In [101]:
from torchtext.datasets import IMDB
from torchtext.vocab import FastText
from torchtext import data, datasets
import torch
import torch.nn as nn
import torch.nn.functional as V
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from IPython.core.debugger import set_trace

In [81]:
train, test = datasets.IMDB.splits(TEXT,LABEL)
TEXT.build_vocab(train, vectors=FastText(language = 'en'))
LABEL.build_vocab(train)

In [88]:
train_iter, test_iter = data.BucketIterator.splits((train, test), batch_size=32, device='cuda',shuffle=True)

In [83]:
batch = next(iter(train_iter))

In [87]:
batch.text[0].shape

torch.Size([128, 998])

In [48]:
weight_matrix = TEXT.vocab.vectors
n_tokens = weight_matrix.size(0)`

In [113]:
class ClassifierModel(nn.Module):
    
    def __init__(self, ntoken, ninp,
                 nhid, nlayers, bsz, noutputs,
                 dropout=0.5):
        super(ClassifierModel, self).__init__()
        self.nhid, self.nlayers, self.bsz = nhid, nlayers, bsz
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.rnn = nn.LSTM(ninp, nhid, nlayers, dropout=dropout)
        self.decoder = nn.Linear(nhid,noutputs)

        self.init_weights()
        self.hidden = self.init_hidden(bsz)
        
    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.uniform_(-initrange, initrange)
 
    def forward(self, input):
        set_trace()
        emb = self.drop(self.encoder(input))
        output, self.hidden = self.rnn(emb, self.hidden)
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1))
 
    def init_hidden(self, bsz):
        weight = next(self.parameters()).data
        return (torch.tensor(weight.new(self.nlayers, bsz, self.nhid).zero_().cuda()),
                torch.tensor(weight.new(self.nlayers, bsz, self.nhid).zero_()).cuda())


weight_matrix = TEXT.vocab.vectors
model = ClassifierModel(weight_matrix.size(0),
weight_matrix.size(1), 200, 3, 32,2)
model = model.cuda()
model.encoder.weight.data.copy_(weight_matrix)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0653, -0.0930, -0.0176,  ...,  0.1664, -0.1308,  0.0354],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       device='cuda:0')

In [119]:
weight_matrix.size(1)


300

In [114]:
model_state = model.state_dict()
pretrained_state = torch.load('lstm_8.pt')
pretrained_state = { k:v for k,v in pretrained_state.items() if k in model_state and v.size() == model_state[k].size() }
model_state.update(pretrained_state)
model.load_state_dict(model_state)

In [115]:
from IPython.core.debugger import set_trace

In [137]:
batch.text[0].shape

torch.Size([32, 820])

In [136]:
for index, batch in enumerate(train_iter):
    pred = model(batch.text[0])

> <ipython-input-113-98151403941a>(24)forward()
     22     def forward(self, input):
     23         set_trace()
---> 24         emb = self.drop(self.encoder(input))
     25         output, self.hidden = self.rnn(emb, self.hidden)
     26         output = self.drop(output)

ipdb> c


RuntimeError: Expected hidden[0] size (3, 820, 200), got (3, 32, 200)